# Imports 

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import importlib

import dgsp
import Bimodularity

# _C. elegans_ bicommunities (Figure 4) (from bimodularity-Figures.ipynb)


In [ ]:
no_sex = False
gap_junc = False

wiring_sym = np.genfromtxt("./data/celegans_graph"+gap_junc*"_GAP"+".csv", delimiter=",")
neuron_df = pd.read_csv("./data/celegans_neurons.csv")

wiring_mod = dgsp.modularity_matrix(wiring_sym, null_model="outin")
print(f"Asymmetric wiring matrix has shape {wiring_sym.shape}")

nodes_labels = neuron_df.loc[:, "Neuron"]
nodes_posx = neuron_df.loc[:, "Position x"]
nodes_posy = neuron_df.loc[:, "Position y"]

d_mat = np.diag(wiring_sym.sum(axis=1))

U, S, Vh = dgsp.sorted_SVD(wiring_mod, fix_negative=False)
V = Vh.T

sort_idx = np.flip(np.argsort(S))
S = S[sort_idx]
U = U[:, sort_idx]
V = V[:, sort_idx]

neuron_df

In [ ]:

graph = wiring_sym

U, S, Vh = dgsp.sorted_SVD(dgsp.modularity_matrix(graph, null_model="outin"))
V = Vh.T

n_nodes = graph.shape[0]

#vector_id_max = 4
vector_id_max = 5
n_kmeans = 5

edge_clusters, edge_clusters_mat = dgsp.edge_bicommunities(graph, U, V, vector_id_max, method="kmeans",
                                                           n_kmeans=n_kmeans, verbose=True, max_k=10)
n_clusters = np.max(edge_clusters)

sending_communities, receiving_communities = dgsp.get_node_clusters(edge_clusters, edge_clusters_mat, method="bimodularity")


# Conjugate matching and tolerance variation

In [ ]:
a = Bimodularity.community_fit_w_tolerance(sending_communities, receiving_communities, 0.5)
#conjugate matching
fits = [(epsilon, Bimodularity.community_fit_w_tolerance(sending_communities, receiving_communities, epsilon)) for epsilon in np.linspace(0, 0.5, 1000)]

# Plotting the results
fig, axes = plt.subplots(2,2, figsize=(15, 6))
                        
axes[0, 0].plot([fit[0] for fit in fits], [len(fit[1][0][0]) for fit in fits], label='match count', color='blue')
axes[0, 0].set_xlabel('tolerance (epsilon)')
axes[0, 0].set_ylabel('match count')
axes[0, 0].set_title('singular community Fit with Tolerance')
axes[0, 0].legend()  
axes[0, 1].plot([fit[0] for fit in fits], [len(fit[1][2][0]) for fit in fits], label='match count', color='red')
axes[0, 1].set_xlabel('tolerance (epsilon)')   
axes[0, 1].set_ylabel('match count')
axes[0, 1].set_title('overall community Fit with Tolerance')
axes[0, 1].legend()
axes[1, 0].plot([fit[0] for fit in fits], [fit[1][1] for fit in fits], label=' total cost', color='blue')
axes[1, 0].set_xlabel('tolerance (epsilon)')
axes[1, 0].set_ylabel('total cost')
axes[1, 0].set_title('singular Total Cost with Tolerance')
axes[1, 0].legend()
axes[1, 1].plot([fit[0] for fit in fits], [fit[1][3] for fit in fits], label=' total cost', color='red')
axes[1, 1].set_xlabel('tolerance (epsilon)')
axes[1, 1].set_ylabel('total cost')
axes[1, 1].set_title('overall Total Cost with Tolerance')
axes[1, 1].legend()
plt.tight_layout()
plt.show()



In [ ]:
gammas = np.linspace(1, 100, 100)

communities = []

for gamma in gammas:
    importlib.reload(dgsp)

    no_sex = False
    gap_junc = False

    wiring_sym = np.genfromtxt("./data/celegans_graph"+gap_junc*"_GAP"+".csv", delimiter=",")
    neuron_df = pd.read_csv("./data/celegans_neurons.csv")

    communities.append(wiring_sym)
    wiring_mod = dgsp.modularity_matrix(wiring_sym, null_model="outin")
    #print(f"Asymmetric wiring matrix has shape {wiring_sym.shape}")

    nodes_labels = neuron_df.loc[:, "Neuron"]
    nodes_posx = neuron_df.loc[:, "Position x"]
    nodes_posy = neuron_df.loc[:, "Position y"]

    d_mat = np.diag(wiring_sym.sum(axis=1))

    U, S, Vh = dgsp.sorted_SVD(wiring_mod, fix_negative=False)
    V = Vh.T

    sort_idx = np.flip(np.argsort(S))
    S = S[sort_idx]
    U = U[:, sort_idx]
    V = V[:, sort_idx]

        
    graph = wiring_sym

    U, S, Vh = dgsp.sorted_SVD(dgsp.modularity_matrix(graph, null_model="outin"))
    V = Vh.T

    n_nodes = graph.shape[0]

    #vector_id_max = 4
    vector_id_max = 5
    n_kmeans = 5

    edge_clusters, edge_clusters_mat = dgsp.edge_bicommunities(graph, U, V, vector_id_max, method="kmeans",
                                                            n_kmeans=n_kmeans, verbose=True, max_k=10)
    n_clusters = np.max(edge_clusters)

    sending_communities, receiving_communities = dgsp.get_node_clusters(edge_clusters, edge_clusters_mat, method="bimodularity")
    sending_communities, receiving_communities = Bimodularity.modify_adjacency_matrix(sending_communities, gamma), Bimodularity.modify_adjacency_matrix(receiving_communities, gamma)
    communities.append((gamma, sending_communities, receiving_communities))




Found 5 clusters !


c:\Users\chris\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=9.
  warnings.warn(
c:\Users\chris\Desktop\MIP Project\Bimodularity\dgsp.py:210: RuntimeWarning: invalid value encountered in divide
  sending_communities / np.sum(edge_clusters_mat > 0, axis=1),
c:\Users\chris\Desktop\MIP Project\Bimodularity\dgsp.py:215: RuntimeWarning: invalid value encountered in divide
  receiving_communities / np.sum(edge_clusters_mat > 0, axis=0),


TypeError: modify_adjacency_matrix() takes 2 positional arguments but 3 were given

In [54]:
for i in communities:
    c= 0
    for j in communities: 
        if np.allclose(i,j):
            c += 1

    print(f" {c} matches")
print(communities[0])
print(communities[2])

 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches
 100 matches

KeyboardInterrupt: 

In [ ]:
# Create heatmap of gamma vs tolerance with match count using pre-computed communities
import seaborn as sns

# Extract gamma values and communities from the pre-computed results
gamma_values = gammas
tolerance_range = np.linspace(0, 0.5, 50)  # More tolerance points for smoother heatmap

print(f"Using {len(communities)} pre-computed gamma values")
print(f"Testing {len(tolerance_range)} tolerance values")

# Initialize arrays to store results
singular_match_counts = np.zeros((len(communities), len(tolerance_range)))
overall_match_counts = np.zeros((len(communities), len(tolerance_range)))

print("Computing tolerance variations for each gamma...")

for i, (gamma, sending_communities, receiving_communities) in enumerate(communities):
    if i % 10 == 0:  # Progress indicator
        print(f"Processing gamma {i+1}/{len(communities)}: {gamma:.3f}")
    
    # Calculate match counts for different tolerance values using pre-computed communities
    for j, tolerance in enumerate(tolerance_range):
        fit_result = Bimodularity.community_fit_w_tolerance(sending_communities, receiving_communities, tolerance)
        singular_match_counts[i, j] = len(fit_result[0][0])  # singular community match count
        overall_match_counts[i, j] = len(fit_result[2][0])   # overall community match count

print("Heatmap computation complete!")

# Create the heatmaps using matplotlib
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Singular community match count heatmap
im1 = axes[0].imshow(singular_match_counts, cmap='viridis', aspect='auto', origin='lower')
axes[0].set_title('Singular Community Match Count\n(Gamma vs Tolerance)')
axes[0].set_xlabel('Tolerance (epsilon)')
axes[0].set_ylabel('Gamma')

# Set tick labels for better readability
n_gamma_ticks = 10
n_tolerance_ticks = 10
gamma_tick_indices = np.linspace(0, len(gamma_values)-1, n_gamma_ticks, dtype=int)
tolerance_tick_indices = np.linspace(0, len(tolerance_range)-1, n_tolerance_ticks, dtype=int)

axes[0].set_xticks(tolerance_tick_indices)
axes[0].set_xticklabels([f'{tolerance_range[i]:.2f}' for i in tolerance_tick_indices])
axes[0].set_yticks(gamma_tick_indices)
axes[0].set_yticklabels([f'{gamma_values[i]:.2f}' for i in gamma_tick_indices])

plt.colorbar(im1, ax=axes[0], label='Match Count')

# Overall community match count heatmap
im2 = axes[1].imshow(overall_match_counts, cmap='viridis', aspect='auto', origin='lower')
axes[1].set_title('Overall Community Match Count\n(Gamma vs Tolerance)')
axes[1].set_xlabel('Tolerance (epsilon)')
axes[1].set_ylabel('Gamma')

axes[1].set_xticks(tolerance_tick_indices)
axes[1].set_xticklabels([f'{tolerance_range[i]:.2f}' for i in tolerance_tick_indices])
axes[1].set_yticks(gamma_tick_indices)
axes[1].set_yticklabels([f'{gamma_values[i]:.2f}' for i in gamma_tick_indices])

plt.colorbar(im2, ax=axes[1], label='Match Count')

plt.tight_layout()
plt.show()

# Create seaborn heatmaps for better aesthetics
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Create DataFrames for seaborn with proper indexing
singular_df = pd.DataFrame(singular_match_counts, 
                          index=[f'{g:.3f}' for g in gamma_values],
                          columns=[f'{t:.3f}' for t in tolerance_range])

overall_df = pd.DataFrame(overall_match_counts,
                         index=[f'{g:.3f}' for g in gamma_values], 
                         columns=[f'{t:.3f}' for t in tolerance_range])

# Seaborn heatmaps with better formatting
sns.heatmap(singular_df, ax=axes[0], cmap='viridis', cbar_kws={'label': 'Match Count'})
axes[0].set_title('Singular Community Match Count\n(Gamma vs Tolerance)')
axes[0].set_xlabel('Tolerance (epsilon)')
axes[0].set_ylabel('Gamma')

# Show only every nth tick for readability
n_display_ticks = 5
gamma_display_ticks = range(0, len(gamma_values), len(gamma_values)//n_display_ticks)
tolerance_display_ticks = range(0, len(tolerance_range), len(tolerance_range)//n_display_ticks)

axes[0].set_xticks(tolerance_display_ticks)
axes[0].set_xticklabels([f'{tolerance_range[i]:.2f}' for i in tolerance_display_ticks])
axes[0].set_yticks(gamma_display_ticks)
axes[0].set_yticklabels([f'{gamma_values[i]:.2f}' for i in gamma_display_ticks])

sns.heatmap(overall_df, ax=axes[1], cmap='viridis', cbar_kws={'label': 'Match Count'})
axes[1].set_title('Overall Community Match Count\n(Gamma vs Tolerance)')
axes[1].set_xlabel('Tolerance (epsilon)')
axes[1].set_ylabel('Gamma')

axes[1].set_xticks(tolerance_display_ticks)
axes[1].set_xticklabels([f'{tolerance_range[i]:.2f}' for i in tolerance_display_ticks])
axes[1].set_yticks(gamma_display_ticks)
axes[1].set_yticklabels([f'{gamma_values[i]:.2f}' for i in gamma_display_ticks])

plt.tight_layout()
plt.show()

# Print some statistics about the results
print(f"\nHeatmap Statistics:")
print(f"Gamma range: {min(gamma_values):.3f} to {max(gamma_values):.3f}")
print(f"Tolerance range: {min(tolerance_range):.3f} to {max(tolerance_range):.3f}")
print(f"Singular match count range: {singular_match_counts.min():.0f} to {singular_match_counts.max():.0f}")
print(f"Overall match count range: {overall_match_counts.min():.0f} to {overall_match_counts.max():.0f}")